# Cat-Dog Classification with Neural Networks - can we reach SOTA?


**Exam:**
Statistical Methods for Machine Learning  
**Author:** 
Edoardo Federici

We will implement two architectures: 
we start with a simple [CNN](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf). Then we'll train a [ViT]() (vision transformer). We will compare results with ConvNets, and then discuss recent advances in CV with the paper about [ConvNeXt]() - 'a convnet for the 2020s', fine-tuning a base (large and xlarge are a no-go) model on the dataset reaching public SOTA on **Cats_vs_Dogs**.

<div class="alert alert-info">

**Note:** After CNN and ViT, the code is in PyTorch for the ConvNeXt (Is from Facebook AI. I've tried loading and fine-tuning the [Tensorflow weights](https://www.tensorflow.org/api_docs/python/tf/keras/applications/convnext/ConvNeXtBase) but they don't perform as good as the PyTorch ones).


_It was done to learn as much as I could, since dealing with multiple libraries seems to be the standard, today._


When dealing with new architectures we often deal with pre-trained models. Pre-training is an expensive task. Recent checkpoints are sadly not all available in PyTorch or Tensorflow. Google side of the Research Community is mostly now using JAX. In a world with giant models and datasets, compute time is an highly valuable resource. Flax reduces training time drastically. It is also 'accelerator indipendent', so it can run on TPUs, GPUs or CPUs without headaces.


_ViTs are pre-trained in Flax, Tensorflow-ported weights may have produced lower scores_
    
</div>

For the experiments we used the _T4_ available on Colab and a _NVIDIA RTX 3060 OC 12GB_.

In [ ]:
!nvidia-smi

Mon Jul 18 01:44:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   50C    P8    19W / 170W |     11MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install tensorflow_addons # for AdamW
!pip install transformers datasets 

In [ ]:
!pip install -q tensorflow_datasets

In [ ]:
import os
import gc 
import numpy as np 
import random
import json
import time
import itertools
import functools 
from statistics import mean
from PIL import Image, UnidentifiedImageError
import glob
import warnings
from typing import Any, Tuple, List, Optional
from dataclasses import dataclass

# progress bar
from tqdm.notebook import tqdm

# tensorflow
import tensorflow_datasets as tfds
import tensorflow as tf

/home/edoardo/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`


# 🔧 **Dataset**

Cats and Dogs dataset from [here](https://unimibox.unimi.it/index.php/s/eNGYGSYmqynNMqF).

- removing faulty images 
- renaming columns etc.

> We have two classes: _**Cat**_ and _**Dog**_

**image_dataset_from_directory** will return a [tf.data.Dataset]() that yields batches of images from the subdirectories classes.

**Faster reading speed?**

Prefetching allow us to read the data for step _s+1_ while the model is computing step _s_. This reduces reading time. 

Caching in memory or on local storage allow us to reduce data reading time. (Using those eats a lot of RAM - which causes fine-tuning on the Keras ConvNeXt / ViT impossible on Colab, skip)

In [ ]:
def dataset_from_directory(path, data_args):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        path,
        validation_split=data_args.train_test_split,
        subset="training",
        color_mode=data_args.mode,
        seed=data_args.seed,
        image_size=data_args.image_size,
        batch_size=data_args.train_batch_size,
        interpolation='bilinear', 
        label_mode=data_args.label
    )

    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        path,
        validation_split=data_args.train_test_split,
        subset="validation",
        color_mode=data_args.mode,
        seed=data_args.seed,
        image_size=data_args.image_size,
        batch_size=data_args.test_batch_size,
        interpolation='bilinear',
        label_mode=data_args.label
    )

    # this causes a memory leak, sadly
    # train_ds = train_ds.cache().prefetch(tf.data.AUTOTUNE)
    # test_ds = test_ds.cache().prefetch(tf.data.AUTOTUNE)
    
    return train_ds, test_ds

**Cleaning pipeline**

Faulty images will cause problems when we train our models. So it's important to clean our folder before. Since we later deal with Torch, we are gonna spot errors with tf.image and with PIL.

In [ ]:
def rm_faulty_images(path: str = None):
    warnings.filterwarnings(
        "ignore", 
        "(Possibly )?corrupt EXIF data", 
        UserWarning
    )

    warnings.filterwarnings(
        "ignore", 
        "extraneous bytes before marker", 
        UserWarning
    )
     
    img_paths = glob.glob(os.path.join(path,'*/*.*'))
    faulty_images = []

    print("Bad paths:")
    for image_path in img_paths:
        try:
            # we deal also with PIL for torch
            # (tf skips jpgs that causes training issues)
            img_bytes = tf.io.read_file(image_path)
            _ = tf.image.decode_image(img_bytes)
            _ = Image.open(image_path)
        except (
            tf.errors.InvalidArgumentError, 
            UnidentifiedImageError
        ) as e:
            print(f"- Found bad path at: {image_path} - {e}")
            faulty_images.append(image_path)

    if len(faulty_images) == 0:
        print("Images are ok!")
    else:
        print("Removing them...")
        for path in faulty_images:
            print(f"- Removing image: {path}")
            os.system(f"rm -rf {path}")

In [ ]:
def cats_vs_dogs(args, get_only=False):
    print(
        " Dataset:    Cats_vs_Dogs\n" 
        " Images:     ~25000\n" 
        f" Image_size: {args.image_size}\n"
        f" Color_mode: {args.mode}\n"
    )

    name = "cats_vs_dogs.zip"
    img_folder = "Cats_vs_Dogs"
    url = "https://unimibox.unimi.it/index.php/s/eNGYGSYmqynNMqF/download"

    target_dir = os.getcwd() + f"/{args.data_path}"
    target = f"{target_dir}/{name}"
    target_imgs = f"{target_dir}/{img_folder}"

    if not os.path.exists(target_dir):
        os.makedirs(target_dir, exist_ok=True)
        print("Downloading...")
        os.system(f"curl {url} -o {target}")
        print(f"Dataset downloaded to {target}")
    
    if not os.path.exists(target_imgs):
        # extract all files
        os.system(f"unzip -qq {target} -d {target_dir}")
        print(f"Dataset extracted!")

        # rename folders to simplify things later 
        os.system(f"mv {target_dir}/CatsDogs {target_imgs}")
        os.system(f"mv {target_imgs}/Cats {target_imgs}/Cat")
        os.system(f"mv {target_imgs}/Dogs {target_imgs}/Dog")

    rm_faulty_images(target_imgs)
    if not get_only:
        train, test = dataset_from_directory(target_imgs, args)
        return train, test

In [ ]:
@dataclass
class DataArguments:
    data_path: str = "dataset"
    image_size: Tuple[int] = (224, 224)
    train_batch_size: int = 32
    test_batch_size: int = 32
    seed: int = 42 # so long, and thanks for all the fish
    train_test_split: float = 0.2
    mode: str = 'rgb'
    label: str = 'categorical'

data_args = DataArguments()

# Dataset for Keras Models

- (_LeNet5_ & _ViT_)

With Keras we use [TensorFlow Datasets](https://www.tensorflow.org/datasets). 

The server is slow, so it'll take some time to load. 

In [ ]:
train_ds, test_ds = cats_vs_dogs(data_args)

In [ ]:
plt.figure(figsize=(10, 10))
class_names = {0: "Cat", 1: "Dog"}

for images, labels in train_ds.skip(1).take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[np.argmax(labels[i].numpy())])
        plt.axis("off")

## **Image Augmentation** and **Normalization**

An important step before starting training is Image Augmentation. When dealing with images (and with text), we can think that:
> more images -> better model

Data Augmentation reduces the chanches of _overfitting_ and is a general best practice when dealing with ConvNets because helps improve generalization. We use several augmentations: 
- **random zoom**
- **random flip**
- **random rotation**

By normalizing the data to a uniform mean of 0 and a standard deviation of 1, faster convergence is achieved. We rescale values from _[0, 1]_ but we can also choose _[-1, 1]_ using:

```
tf.keras.layers.Rescaling(scale=1./127.5, offset=-1)
```
We apply augmentation only on the train dataset, we don't want all those changes for evaluation (we apply only a small horizontal flip). 

If you're using Tensorflow version 2.9.1 you can also use random brightness etc. 



In [ ]:
train_pipeline = tf.keras.Sequential(
    [   
        tf.keras.layers.RandomZoom(
            height_factor=0.4, width_factor=0.4
        ),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1), 
        tf.keras.layers.Normalization(mean=0., variance=1.),  
        tf.keras.layers.Rescaling(scale=1./255),   
    ]
)

test_pipeline = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.Normalization(mean=0., variance=1.),
        tf.keras.layers.Rescaling(scale=1./255),
    ]
)

Let’s now see how random augmentation affect images from the training set.

In [ ]:
plt.figure(figsize=(10, 10))
class_names = {0: "Cat", 1: "Dog"}

for images, labels in train_ds.skip(1).take(1):
    images = train_pipeline(images)
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow((images[i].numpy()*255).astype("uint8"))
        plt.title(class_names[np.argmax(labels[i].numpy())])
        plt.axis("off")

## _K-Fold_

We use K-Fold Cross-Validation. Our Train set is splitted into n-folds. One fold is used as Validation, remaining folds are joined to create a i-th train set. It's not the cleanest from-scratch implementation but it does the job. 

In [ ]:
# partially adapted from 
# https://github.com/fenwickslab/fenwicks/blob/2d539634efe829bd5a2e2c1b9c837819a89bb707/data.py#L370

class KFold:
    def __init__(
        self, 
        dataset, 
        n_folds: Optional[int] = 5, 
    ):
        self.dataset = dataset
        self.n_folds = n_folds
        self.ind = iter(range(n_folds+1))

    def get_fold(self, val_fold_idx, training):
        if training:
            trn_shards = itertools.chain(
                range(val_fold_idx), 
                range(val_fold_idx + 1, self.n_folds)
            )
            update_func = lambda ds, i: ds.concatenate(self.dataset.shard(self.n_folds, i))
            dataset = functools.reduce(
                update_func, 
                trn_shards, 
                self.dataset.shard(self.n_folds, next(trn_shards))
            )
        else:
            dataset = self.dataset.shard(self.n_folds, val_fold_idx)
        return dataset

    def __getitem__(self, _):
        index = next(self.ind)
        train = self.get_fold(index, True)
        val = self.get_fold(index, False)
        return train, val

# 🕰 **LeNet5** 

Lenet-5 is one of the earliest pre-trained models proposed by Yann LeCun and others in the year 1998, in the research paper [Gradient-Based Learning Applied to Document Recognition](). They used this architecture for recognizing the handwritten and machine-printed characters.

The main reason behind the popularity of this model was its simple and straightforward architecture. It is a multi-layer convolution neural network for image classification.

- three sets of **convolutions**
- two **dense layers**

Paper: [Gradient-Based Learning Applied to Document Recognition]()

Abstract: _**Multilayer neural networks trained with the back-propagation algorithm constitute the best example of a successful gradient based learning technique**. Given an appropriate network architecture, gradient-based learning algorithms can be used to synthesize a complex decision surface that can classify high-dimensional patterns, such as handwritten characters, with minimal preprocessing. This paper reviews various methods applied to handwritten character recognition and compares them on a standard handwritten digit recognition task. **Convolutional neural networks, which are specifically designed to deal with the variability of 2D shapes, are shown to outperform all other techniques**._

In [ ]:
print("Tensorflow:", tf.__version__)

from sklearn.metrics import zero_one_loss
from datetime import datetime 
from keras.layers import (
    Dense, 
    Flatten, 
    Conv2D, 
    MaxPooling2D,
    Dropout, 
    BatchNormalization
) 
import keras as k
from keras import layers
from keras import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow_addons.optimizers import AdamW

In [ ]:
@dataclass
class LeNetTrainingArgs:
    model_name_or_path: str = "lenet"
    monitor_best: str = "val_loss"
    epochs: int = 10
    n_folds: int = 5
    learning_rate: float = 0.001
    dropout: float = 0.2
    jit_compile: bool = False
    log_dir: str = "lenet/logdir/"

ln_args = LeNetTrainingArgs()

In [ ]:
kfold = KFold(train_ds, n_folds=ln_args.n_folds)

This is a '_modernized_' version of LeNet:

- **TanH** is replaced with **ReLU**
- ReLU exploding gradients can be solved with **He initialization**
- **Dropout** was added before the bigger dense layer, which helps reducing overfitting 
- **Batch Normalization** was added

In [ ]:
class LeNet(Model):
    def __init__(self, 
            dropout: float = 0.4, 
            image_size: int = 224, 
            num_ch: int = 3
    ):
        super(LeNet, self).__init__()

        self.conv1 = Conv2D(
            32, 5, padding="same",
			input_shape=(image_size, image_size, num_ch), 
            activation='relu', 
            kernel_initializer='he_uniform'
        )
        self.max_pool1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))
        self.batch_norm1 = BatchNormalization()

        self.conv2 = Conv2D(
            64, 5, padding="valid", 
            activation='relu', 
            kernel_initializer='he_uniform'
        )
        self.max_pool2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))
        self.batch_norm2 = BatchNormalization()

        self.conv3 = Conv2D(
            64, 5, padding="valid", 
            activation='relu', 
            kernel_initializer='he_uniform'
        )
        self.max_pool3 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))
        self.batch_norm3 = BatchNormalization()

        # this produces a vector |v| = 1600
        self.flatten = Flatten()
        self.dropout = Dropout(dropout)

        # dense layers for classification
        self.d1 = Dense(256, activation='relu', kernel_initializer='he_uniform')
        self.d2 = Dense(2, activation='softmax')
    
    def call(self, x, training=False):
        x = self.max_pool1(self.conv1(x))
        x = self.batch_norm1(x)

        x = self.max_pool2(self.conv2(x))
        x = self.batch_norm2(x)

        x = self.max_pool3(self.conv3(x))
        x = self.batch_norm3(x)

        x = self.flatten(x)
        if training:
            x = self.dropout(x, training=training)
        x = self.d1(x)
        x = self.d2(x)
        return x

In [ ]:
loss_fn = k.losses.CategoricalCrossentropy()

train_acc = k.metrics.CategoricalAccuracy()
test_acc = k.metrics.CategoricalAccuracy()

***Choose the optimizer:***

In [ ]:
# adamw
def get_model():
    lr_schedule = tf.optimizers.schedules.ExponentialDecay(1e-3, 100, 0.9)
    wd_schedule = tf.optimizers.schedules.ExponentialDecay(5e-5, 100, 0.9)

    opt = AdamW(learning_rate=lr_schedule, weight_decay=lambda: None)
    opt.weight_decay = lambda: wd_schedule(opt.iterations)

    model = LeNet(
        dropout = ln_args.dropout, 
        image_size = data_args.image_size[0], 
        num_ch = 3
    )

    return model, opt

In [ ]:
# sgd momentum 
def get_model():
    lr_schedule = tf.optimizers.schedules.ExponentialDecay(1e-3, 100, 0.9)

    opt = tf.keras.optimizers.SGD(
        learning_rate=lr_schedule, momentum=0.9, nesterov=False, name="SGD"
    )

    model = LeNet(
        dropout = ln_args.dropout, 
        image_size = data_args.image_size[0], 
        num_ch = 3
    )

    return model, opt

In [ ]:
# adam
def get_model():
    lr_schedule = tf.optimizers.schedules.ExponentialDecay(1e-3, 100, 0.9)

    opt = tf.keras.optimizers.Adam(
        learning_rate=lr_schedule,
    )

    model = LeNet(
        dropout = ln_args.dropout, 
        image_size = data_args.image_size[0], 
        num_ch = 3
    )

    return model, opt

**Train and Test step:**

In [ ]:
def train_step(images, labels):
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        loss_value = loss_fn(labels, logits)

    grads = tape.gradient(loss_value, model.trainable_weights)
    opt.apply_gradients(zip(grads, model.trainable_weights))
    train_acc.update_state(labels, logits)

    return loss_value

def test_step(images, labels):
    val_logits = model(images, training=False)
    loss_value = loss_fn(labels, val_logits)
    test_acc.update_state(labels, val_logits)

    predicted = np.argmax(val_logits, axis=-1)
    labels = np.argmax(labels, axis=-1)
    zo_loss = zero_one_loss(labels, predicted)

    return zo_loss, loss_value.numpy()

**Training loop**: 

In [ ]:
num_train_examples = train_ds.cardinality().numpy()
steps_per_epoch = num_train_examples // ((ln_args.n_folds-1) * data_args.train_batch_size)
epochs = ln_args.epochs

print("Running training:")
print(f"Total examples from train_ds = {num_train_examples}")
print(f"Num Epochs for each fold = {epochs}")
print(f"Train batch size = {data_args.train_batch_size}")
print(f"Num folds = {ln_args.n_folds}\n")

for fold, (train_fold, test_fold) in enumerate(kfold):
    num_fold_examples = train_fold.cardinality().numpy()
    print(f"\nFold: {fold+1} ", "="*10)
    print(f"Total fold examples: {num_fold_examples}")

    # ======== Create Model ==========
    model, opt = get_model()

    epochs_progress_bar = tqdm(range(epochs), desc=f"Epoch: (1/{epochs})", position=1)
    val_loss_global = float("inf")

    for epoch in range(epochs):
        start_time = time.time()
        print(f"\nEpoch: {epoch+1}")

        # ======================== Training ===============================
        train_step_progress_bar = tqdm(total=num_fold_examples, desc="Training", position=2, leave=True)
        for step, (images, labels) in enumerate(train_fold):
            images = train_pipeline(images)
            loss_value = train_step(images, labels)
            desc = (
                f"Training loss at step {step}: {round(float(loss_value), 4)}\n"
            )
            gc.collect()
            train_step_progress_bar.desc = desc
            train_step_progress_bar.update(1)  

        train_accuracy = train_acc.result()
        print(f"Training acc over epoch: {round(float(train_accuracy), 4)}")
        train_acc.reset_states()

        # ======================== Evaluation ==============================
        zo_val_loss, val_loss = [], []
        for images, labels in test_fold:
            images = test_pipeline(images)
            zo_val_loss_step, val_loss_step = test_step(images, labels)
            zo_val_loss.append(zo_val_loss_step)
            val_loss.append(val_loss_step)
        zo_val_loss, val_loss = mean(zo_val_loss), mean(val_loss)

        print("Zero-One: ", zo_val_loss)
        print("Val loss: ", val_loss)

        val_accuracy = test_acc.result()
        test_acc.reset_states()

        if val_loss < val_loss_global:
            print(f"Validation loss decreased: {round(val_loss, 4)}")
            val_loss_global = val_loss
            model.save(
                f"{ln_args.model_name_or_path}/lenet_epoch_{epoch}_fold_{fold}"
            )

        desc = (
            f"Validation acc: {round(float(val_accuracy), 4)} | "
            f"Time taken: {round(time.time() - start_time, 4)}"
        )

        # ======================== Housekeeping ============================
        gc.collect()
        k.backend.clear_session()

        epochs_progress_bar.write(desc)
        epochs_progress_bar.desc = desc
        epochs_progress_bar.update(1) 

# 📱 Keras **ViT** fine-tuning using HuggingFace

Easy implementation with Keras, training it with built-in model.fit

Here we don't perform Cross-Validation, it would take too much time to perform several epochs so many times.

Paper: [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929)

Abstract: _**While the Transformer architecture has become the de-facto standard for natural language processing tasks, its applications to computer vision remain limited.** In vision, attention is either applied in conjunction with convolutional networks, or used to replace certain components of convolutional networks while keeping their overall structure in place. **We show that this reliance on CNNs is not necessary and a pure transformer applied directly to sequences of image patches can perform very well on image classification tasks. When pre-trained on large amounts of data and transferred to multiple mid-sized or small image recognition benchmarks (ImageNet, CIFAR-100, VTAB, etc.), Vision Transformer (ViT) attains excellent results compared to state-of-the-art convolutional networks while requiring substantially fewer computational resources to train**._

In [ ]:
!pip install transformers 

In [ ]:
from tensorflow.keras.callbacks import TensorBoard as TensorboardCallback, EarlyStopping
from transformers import TFViTModel
from tensorflow_addons.optimizers import AdamW

In [ ]:
@dataclass
class ViTrainingArgs:
    model_name_or_path: str = "google/vit-base-patch16-224-in21k"
    monitor_best: str = "val_loss"
    epochs: int = 5
    log_dir: str = "vit/logdir/"
    num_labels: int = 2

vit_args = ViTrainingArgs()

We could define our network as previously done, inheriting from Keras Model. Keras allow us to define our NN with a more compact form (litterally the same thing in Flax - which is JAX based, the framework in which ViTs were originally trained - is called [@nn.compact](). We can go through model layers and pass our data through them. It will automatically create the computation graph. Since it was trained with images in a channel_first flavour, we need to permute our channel_last images before feeding the network.

```python
class ViTImageClassifier(Model):
    def __init__(
        self, 
        name: str
    ):
        self.permute = tf.keras.layers.Permute((3, 1, 2))
        self.pretrained = TFViTModel.from_pretrained(
            name,
            add_pooling_layer=False
        )
        self.classifier = tf.keras.layers.Dense(
            units=vit_args.num_labels,
            kernel_initializer="he_uniform",
            activation='softmax',
            name="classifier"
        )
    
    def call(self, x):
        x = self.permute(x)
        x = self.pretrained.vit(x)[0]
        x = self.classifier(x[:, 0, :])
        return x
```

This can be written as:



In [ ]:
base_model = TFViTModel.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    add_pooling_layer=False
)

input = tf.keras.layers.Input(shape=(224, 224, 3), name='pixel_values', dtype='float32')
channel_first = tf.keras.layers.Permute((3, 1, 2))(input)
vit = base_model.vit(channel_first)[0]
classifier = tf.keras.layers.Dense(
    units=vit_args.num_labels,
    kernel_initializer="he_uniform",
    activation='softmax',
    name="classifier"
)(vit[:, 0, :])

model = tf.keras.Model(inputs=input, outputs=classifier)

In [ ]:
callbacks = [
    TensorboardCallback(log_dir=vit_args.log_dir)
]

In [ ]:
num_train_steps = train_ds.cardinality().numpy() * vit_args.epochs

lr_schedule = tf.optimizers.schedules.ExponentialDecay(1e-4, 100, 0.9)
wd_schedule = tf.optimizers.schedules.ExponentialDecay(5e-5, 100, 0.9)

optimizer = AdamW(learning_rate=lr_schedule, weight_decay=lambda: None)
optimizer.weight_decay = lambda: wd_schedule(optimizer.iterations)

loss = tf.keras.losses.CategoricalCrossentropy()

accuracy = tf.keras.metrics.CategoricalAccuracy()

model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = accuracy
)

Images values must be in the range _[-1, 1]_ for ViT. So the Rescaling layer is different here. 

In [ ]:
train_pipeline = tf.keras.Sequential(
    [   
        tf.keras.layers.RandomZoom(
            height_factor=0.4, width_factor=0.4
        ),
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.04), 
        tf.keras.layers.Normalization(mean=0., variance=1.),
        tf.keras.layers.Rescaling(scale=1./127.5, offset=-1)  
    ]
)

test_pipeline = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.Normalization(mean=0., variance=1.),
        tf.keras.layers.Rescaling(scale=1./127.5, offset=-1)
    ]
)

In [ ]:
vit_train_ds = train_ds.map(
    lambda x, y: (
        train_pipeline(x, training=True), y), 
    num_parallel_calls=tf.data.AUTOTUNE)

vit_test_ds = test_ds.map(
    lambda x, y: (
        test_pipeline(x, training=False), y), 
    num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
#@title Remove ViT Folder and models
#@markdown If you need to clean up output folder.
!rm -rf vit/

In [ ]:
#@title ViT Tensorboard 
%reload_ext tensorboard
%tensorboard --logdir vit/logdir/

In [ ]:
history = model.fit(
    vit_train_ds,
    validation_data=vit_test_ds,
    callbacks=callbacks,
    epochs=vit_args.epochs,
)

# 🎺 **ConvNeXt** 

We're gonna implement **step-by-step** a ConvNeXt model and train it on _Cats_vs_Dogs_, using pre-trained weights.

Paper: https://arxiv.org/abs/2201.03545

Abstract: _**The “Roaring 20s” of visual recognition began with the
introduction of Vision Transformers (ViTs), which quickly
superseded ConvNets as the state-of-the-art image classification model**. A vanilla ViT, on the other hand, faces difficulties
when applied to general computer vision tasks such as object
detection and semantic segmentation. **It is the hierarchical
Transformers (e.g., Swin Transformers) that reintroduced several ConvNet priors, making Transformers practically viable
as a generic vision backbone and demonstrating remarkable
performance on a wide variety of vision tasks**. However,
the effectiveness of such hybrid approaches is still largely
credited to the intrinsic superiority of Transformers, rather
than the inherent inductive biases of convolutions. In this
work, we reexamine the design spaces and test the limits of
what a pure ConvNet can achieve. **We gradually “modernize”
a standard ResNet toward the design of a vision Transformer,
and discover several key components that contribute to the
performance difference along the way. The outcome of this
exploration is a family of pure ConvNet models dubbed ConvNeXt**. Constructed entirely from standard ConvNet modules,
ConvNeXts compete favorably with Transformers in terms of
accuracy and scalability, **achieving 87.8% ImageNet top-1
accuracy and outperforming Swin Transformers on COCO
detection and ADE20K segmentation, while maintaining the
simplicity and efficiency of standard ConvNets**._

In [ ]:
%%capture
!pip install split-folders
!pip install torch_optimizer

In [ ]:
import torch 
import torch.nn as nn 
import splitfolders
import torch.nn.functional as F
from torch.utils.data import DataLoader
from typing import List, Optional, Tuple
from datasets import load_dataset, load_metric

In [ ]:
@dataclass
class ConvNeXtDataArguments:
    input_folder: str = 'dataset/Cats_vs_Dogs'
    dataset_folder: str = 'dataset/Cat_vs_Dogs_split'
    split_ratio: Tuple[float] = (0.9, 0.1)
    seed: int = 1337
    num_labels: int = 2

cn_args = ConvNeXtDataArguments()

In [ ]:
cats_vs_dogs(data_args, get_only=True)

In [ ]:
splitfolders.ratio(
    cn_args.input_folder, 
    output=cn_args.dataset_folder, 
    seed=cn_args.seed, 
    ratio=cn_args.split_ratio
)

In [ ]:
#@title Check
train_cats = next(os.walk(cn_args.dataset_folder + "/train/Cat"))[2]
train_dogs = next(os.walk(cn_args.dataset_folder + "/train/Dog"))[2]
val_cats = next(os.walk(cn_args.dataset_folder + "/val/Cat"))[2]
val_dogs = next(os.walk(cn_args.dataset_folder + "/val/Dog"))[2]
print("Train Cats: ", len(train_cats))
print("Train Dogs: ", len(train_dogs))
print("Val Cats: ", len(val_cats))
print("Val Dogs: ", len(val_dogs))

## ⚛ Model

We can find model implementations in HuggingFace [Transformers](https://github.com/huggingface/transformers) or in [Timm](https://github.com/rwightman/pytorch-image-models) (Pytorch Image Models) from Ross Wightman. This implementation is heavily inspired from those, but we clean the code reducing possible clutter. We'll go step by step explaining all the sections. 


In [ ]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

data_path = "./dataset/Cats_vs_Dogs/"

transform_img = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

image_data = torchvision.datasets.ImageFolder(
  root=data_path, transform=transform_img
)

image_data_loader = DataLoader(
  image_data, 
  batch_size=32, 
  shuffle=False
)


In [ ]:
mean = []
std = []

for inputs, labels in tqdm(image_data_loader):
    mean.append(inputs.mean([0,2,3]).numpy())
    std.append(inputs.std([0,2,3]).numpy())
    
print("Mean: ", np.mean(mean, axis=0))
print("Std: ", np.mean(std, axis=0))

**If training from scratch, use cats_vs_dogs, if fine-tuning use imagenet**

In [ ]:
# catsvsdogs
preprocessor_config = {
    "crop_pct": 0.875,
    "do_normalize": True,
    "do_resize": True,
    "image_mean": [0.49034384, 0.45506233, 0.4163233],
    "image_std": [0.2585075, 0.2513996, 0.25332677],
    "resample": 3,
    "size": 224
}

In [ ]:
# imagenet1k
preprocessor_config = {
    "crop_pct": 0.875,
    "do_normalize": True,
    "do_resize": True,
    "image_mean": [0.485, 0.456, 0.406],
    "image_std": [
        0.229,
        0.224,
        0.225
    ],
    "resample": 3,
    "size": 224
}

Pre-trained model is hosted on Huggingface.

In [ ]:
#@title Choose pre-trained
model_name = "convnext-base-224-22k" #@param ["convnext-base-224-22k-1k", "convnext-base-224-22k", "convnext-tiny-224"]
url = f"https://huggingface.co/facebook/{model_name}"

os.system("git lfs install")
print("Downloading the model...")
_ = os.system(f"git clone {url}")
print("Model downloaded!")

>**Stochastic Depth**

**DropPath** (Stochastic Depth) regularization layers: https://arxiv.org/abs/1603.09382. It is adapted from the _timm_ repository.

Regularization often yields better generalization properties by avoiding overfitting. 

From the paper:
_**Very deep convolutional networks with hundreds of layers have led to significant reductions in error on competitive benchmarks**. Although the unmatched expressiveness of the many layers can be highly desirable at test time, training very deep networks comes with its own set of challenges. **The gradients can vanish, the forward flow often diminishes, and the training time can be painfully slow. To address these problems, we propose stochastic depth, a training procedure that enables the seemingly contradictory setup to train short networks and use deep networks at test time**. We start with very deep networks but during training, for each mini-batch, randomly drop a subset of layers and bypass them with the identity function. This simple approach complements the recent success of residual networks. **It reduces training time substantially and improves the test error significantly on almost all data sets that we used for evaluation**. With stochastic depth we can increase the depth of residual networks even beyond 1200 layers and still yield meaningful improvements in test error (4.91% on CIFAR-10)_

In [ ]:
class DropPath(nn.Module):
    """
    Stochastic depth
    """

    def __init__(self, drop_prob=None):
        super().__init__()
        self.p = drop_prob

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if not self.training or self.p == 0.0:
            return x

        keep_prob = 1. - self.p
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        random_tensor = keep_prob + torch.rand(*shape, dtype=x.dtype).to(x.device)
        random_tensor.floor_()  # binarize
        x = x.div(keep_prob) * random_tensor
        return x

>**Layer Normalization** 

From the paper we read that they replaced every Batch Normalization layer with Layer Normalization. 

From the paper:
_BatchNorm is an essential component in ConvNets as it improves the convergence
and reduces overfitting. However, BN also has many intricacies that can have a detrimental effect on the model’s
performance. There have been numerous attempts at
developing alternative normalization techniques,
but BN has remained the preferred option in most vision
tasks. On the other hand, the simpler Layer Normalization (LN) has been used in Transformers, resulting in
good performance across different application scenarios.
**Directly substituting LN for BN in the original ResNet
will result in suboptimal performance**. With all the modifications in network architecture and training techniques,
here we revisit the impact of using LN in place of BN. **We
observe that our ConvNet model does not have any difficulties training with LN; in fact, the performance is slightly
better, obtaining an accuracy of 81.5%**.
From now on, we will use one LayerNorm as our choice
of normalization in each residual block._

Layer Normalization: https://arxiv.org/abs/1607.06450

How it works?

$$y_i=\lambda(\frac{x_i-\mu}{\sqrt{\sigma^2+\epsilon}})+\beta$$

Gamma and Beta are two parameters, Epsilon is a constant. Specifically, given a sample of shape [N, C, H, W] LayerNorm calculates a mean and variance of all the elements of shape [C, H, W] in each batch. This method not only solves both problems mentioned above, but also removes the requirement for storing mean and variances for inference (something which Batch Normalization layers needs to do during training).

In [ ]:
class LayerNorm(nn.Module):
    """
    LayerNorm is a normalization layer:
    it normalizes the activations of the previous layer for each example.
    """

    def __init__(self, features, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(features))
        self.bias = nn.Parameter(torch.zeros(features))
        self.eps = eps
        self.features = (features,)
        self.data_format = data_format

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.data_format == "channels_last":
            x = F.layer_norm(x, self.features, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
        return x

From Swim Transformer. Images are splitted in patches to feed the network.  

In [ ]:
class Embeddings(nn.Module):
    """
    This class is comparable to (and inspired by) the SwinEmbeddings class.
    """

    def __init__(self, config):
        super().__init__()
        self.patch_embeddings = nn.Conv2d(
            config.num_channels,
            config.hidden_sizes[0],
            kernel_size=config.patch_size,
            stride=config.patch_size,
        )
        self.layer_norm = LayerNorm(config.hidden_sizes[0], eps=1e-6, data_format="channels_first")

    def forward(self, x: torch.FloatTensor) -> torch.Tensor:
        """
        :param x: pixel_values (batch_size, height, width, num_channels)
        :return: embeddings (batch_size, height, width, hidden_size)
        """
        x = self.patch_embeddings(x)
        x = self.layer_norm(x)
        return x

They used larger kernel sizes _(7x7)_.

From the paper:
_We experimented with several kernel sizes, including 3, 5, 7, 9, and 11. The network’s performance increases
from 79.9% (3×3) to 80.6% (7×7), while the network’s
FLOPs stay roughly the same._

In [ ]:
class ConvNeXtLayer(nn.Module):
    """
    This is a Block of the ConvNeXt model.

    - [DwConv, Permute to (N, H, W, C), LayerNorm (channels_last), Linear, GELU, Linear]; Permute back

    This implementation according to HuggingFace's engineers is faster.
    """

    def __init__(self, config, dim, drop_path=0):
        super().__init__()
        self.dw_conv = nn.Conv2d(
            dim,
            dim,
            kernel_size=7,
            padding=3,
            groups=dim,
        )
        self.layer_norm = LayerNorm(dim, eps=1e-6)
        self.pw_conv1 = nn.Linear(dim, dim * 4)
        self.act = nn.GELU()
        self.pw_conv2 = nn.Linear(dim * 4, dim)
        self.layer_scale = nn.Parameter(
            config.layer_scale_init_value * torch.ones((dim)), requires_grad=True
        )

        self.drop_path = DropPath(drop_prob=drop_path) if drop_path else nn.Identity()

    def forward(self, x: torch.FloatTensor) -> torch.Tensor:
        """
        :param x: (N, C, H, W), we convert it in (N, H, W, C)
        :return: (N, C, H, W)
        """
        input = x
        x = self.dw_conv(x)
        # (N, H, W, C)
        x = x.permute(0, 2, 3, 1)  
        x = self.layer_norm(x)
        x = self.pw_conv1(x)
        x = self.act(x)
        x = self.pw_conv2(x)
        if self.layer_scale is not None:
            x = x * self.layer_scale

        # repermute to (N, C, H, W)
        x = x.permute(0, 3, 1, 2)

        x = input + self.drop_path(x)
        return x

In [ ]:
class ConvNeXtStage(nn.Module):
    """
    This is a Stage of the ConvNeXt model.
    We add an optional downsampling step and multiple residual blocks.
    """

    def __init__(
        self,
        config,
        in_channels,
        out_channels,
        kernel_size=2,
        stride=2,
        depth=2,
        drop_path_rates: List[float] = None,
    ):
        super().__init__()
        if in_channels != out_channels or stride > 1:
            self.downsample = nn.Sequential(
                LayerNorm(in_channels, eps=1e-6, data_format="channels_first"),
                nn.Conv2d(
                    in_channels,
                    out_channels,
                    kernel_size=kernel_size,
                    stride=stride,
                ),
            )
        else:
            self.downsample = nn.Identity()

        self.layers = nn.ModuleList(
            [
                ConvNeXtLayer(
                    config, out_channels, drop_path=drop_path_rates[i]
                ) for i in range(depth)
            ]
        )

    def forward(self, x: torch.FloatTensor) -> torch.Tensor:
        x = self.downsample(x)
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
class ConvNeXtEncoder(nn.Module):
    """
    Hi I'm the ConvNeXt Encoder.
    """

    def __init__(self, config):
        super().__init__()
        drop_path_rates = [
            x.tolist()
            for x in torch.linspace(0, config.drop_path_rate, sum(config.depths)).split(
                config.depths
            )
        ]

        self.stages = nn.ModuleList()
        previous_channels = config.hidden_sizes[0]

        for i in range(config.num_stages):
            out_channels = config.hidden_sizes[i]
            stage = ConvNeXtStage(
                config,
                in_channels=previous_channels,
                out_channels=out_channels,
                drop_path_rates=drop_path_rates[i],
                stride=2 if i > 0 else 1,
                depth=config.depths[i]
            )
            self.stages.append(stage)
            previous_channels = out_channels

    def forward(self, x: torch.FloatTensor) -> torch.FloatTensor:
        for layer in self.stages:
            x = layer(x)
        return x

In [ ]:
class ConvNeXtModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.embeddings = Embeddings(config)
        self.encoder = ConvNeXtEncoder(config)

        self.layernorm = nn.LayerNorm(config.hidden_sizes[-1], eps=config.layer_norm_eps)

    def forward(
        self,
        x: torch.FloatTensor = None
    ):
        if x is None:
            raise ValueError("You have to specify pixel_values")

        embedding_output = self.embeddings(x)
        encoder_outputs = self.encoder(
            embedding_output
        )

        o_pooled = self.layernorm(encoder_outputs.mean([-2, -1]))
        return o_pooled

Let's define our Classifier. Add a Linear Layer on top to perform classification. 

In [ ]:
class ImageClassifier(nn.Module):
    """
    This is the ConvNext model for image classification.
    """

    def __init__(self, config, num_labels):
        super().__init__()

        self.num_labels = num_labels
        self.convnext = ConvNeXtModel(config)
        self.classifier = nn.Linear(config.hidden_sizes[-1], self.num_labels)

    def forward(
        self,
        pixel_values: torch.FloatTensor,
        labels: Optional[torch.LongTensor] = None
    ) -> torch.FloatTensor:

        x = self.convnext(pixel_values)
        logits = self.classifier(x)
        return logits

In [ ]:
@dataclass
class ConvNeXtTrainingArgs:
    epochs = 50
    learning_rate = 1e-3
    warmup_steps = 0
    weight_decay = 1e-8

cn_train = ConvNeXtTrainingArgs()

## Pre-trained weights 🙀🔥
Here we load pre-trained weights adapting them for our model. 

In [ ]:
class clsprops:
	def __init__(self, data):
		self.__dict__ = data

In [ ]:
class ConvNeXtClassifier:
    """
    A class to load pretrained checkpoints into our model
    """
    @classmethod
    def clean_state_dict(cls, model, state_dict):
        model_fields = set(k for k in model.state_dict().keys())
        model_state_dict = model.state_dict()
        hf_field_names = state_dict.keys()
        unused_fields = set(hf_field_names)
        remap = {}

        mismatched_fields = []
        for field_name in hf_field_names:
            new_name = field_name.replace("pwconv", "pw_conv")
            new_name = new_name.replace("dwconv", "dw_conv")
            new_name = new_name.replace("layernorm", "layer_norm")
            new_name = new_name.replace("downsampling_layer", "downsample")
            new_name = new_name.replace("layer_scale_parameter", "layer_scale")
            new_name = new_name.replace("layernorm", "layer_norm")

            # since it is from nn it is called layernorm 
            new_name = new_name.replace("convnext.layer_norm.bias", "convnext.layernorm.bias")
            new_name = new_name.replace("convnext.layer_norm.weight", "convnext.layernorm.weight")

            if new_name in model_fields:
                # cleans last linear layer, if there's a mismatch, skip
                # we don't use the ImageNet pretrained head
                if state_dict[field_name].shape != model_state_dict[new_name].shape:
                    mismatched_fields.append(new_name)
                else:
                    model_fields.remove(new_name)
                    unused_fields.remove(field_name)
                    remap[new_name] = state_dict[field_name]
            
        model.load_state_dict(remap, strict=False)
        return model_fields, unused_fields, mismatched_fields

    @classmethod
    def from_pretrained(cls, checkpoint_dir: str, map_location=None, num_labels=2, **kwargs):
        if os.path.isdir(checkpoint_dir):
            checkpoint = os.path.join(checkpoint_dir, 'pytorch_model.bin')
            config = json.load(open(os.path.join(checkpoint_dir, 'config.json')))
            config = clsprops(config)
            
        else:
            checkpoint = checkpoint_dir
        hf_dict = torch.load(checkpoint, map_location=map_location)

        model = ImageClassifier(config, num_labels=num_labels)
        mfn, ucf, msf = ConvNeXtClassifier.clean_state_dict(model, hf_dict)

        print("Pre-trained ConvNeXt loaded! 🔥")
        print("Unused pre-trained fields: ", len(ucf))
        print("Pretrained Head weights not loaded: ", msf)
        return model
    
    @classmethod
    def from_scratch(cls, checkpoint_dir: str, map_location=None, num_labels=2, **kwargs):
        if os.path.isdir(checkpoint_dir):
            config = json.load(open(os.path.join(checkpoint_dir, 'config.json')))
            config = clsprops(config)
            config.drop_path_rate = 0.3

        model = ImageClassifier(config, num_labels=num_labels)
        
        def _init_weights(module):
            if isinstance(module, (nn.Linear, nn.Conv2d)):
                module.weight.data.normal_(mean=0.0, std=config.initializer_range)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.LayerNorm):
                module.bias.data.zero_()
                module.weight.data.fill_(1.0)

        model.apply(_init_weights)
        print("From-scrach ConvNeXt! 🔥")
        return model

**choose your model:** 

if you want to fine-tune choose from_pretrained, else from_scratch

In [ ]:
model = ConvNeXtClassifier.from_pretrained(f"./{model_name}", num_labels=2)

In [ ]:
model = ConvNeXtClassifier.from_scratch(f"./{model_name}", num_labels=2)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

## : Dataset 

We can, from Tf data, convert images to numpy, convert them into torch Tensors, but it sounds crazy and it's more practical to use torch functions, that make our life a little easier. 

In [ ]:
train_dir = cn_args.dataset_folder + "/train"
val_dir = cn_args.dataset_folder + "/val"

In [ ]:
data_files = {}
data_files["train"] = os.path.join(train_dir, "**")
data_files["validation"] = os.path.join(val_dir, "**")

dataset = load_dataset(
    "imagefolder",
    data_files=data_files,
    task="image-classification",
)

In [ ]:
dataset

In [ ]:
labels = dataset["train"].features["labels"].names

label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
print(f"Labels: {labels}")
print(f"Id of Cat: {label2id['Cat']}")
print(f"Id of Dog: {label2id['Dog']}")

**Image Augmentation and Normalization**

As previously done, we apply image augmentation and normalization.

In [ ]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

In [ ]:
normalize = Normalize(mean=preprocessor_config["image_mean"], std=preprocessor_config["image_std"])
_train_transforms = Compose(
    [
        RandomResizedCrop(preprocessor_config["size"]),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)
_val_transforms = Compose(
    [
        Resize(preprocessor_config["size"]),
        CenterCrop(preprocessor_config["size"]),
        ToTensor(),
        normalize,
    ]
)

def train_transforms(example_batch):
    example_batch["pixel_values"] = [
        _train_transforms(pil_img.convert("RGB")) for pil_img in example_batch["image"]
    ]
    return example_batch

def val_transforms(example_batch):
    example_batch["pixel_values"] = [_val_transforms(pil_img.convert("RGB")) for pil_img in example_batch["image"]]
    return example_batch

In [ ]:
# Dataset.set_transform() allows you to apply a custom formatting transform on-the-fly.
# https://huggingface.co/docs/datasets/process
# Transforms are applied only when the images are accessed.

dataset["train"].set_transform(train_transforms)
dataset["validation"].set_transform(train_transforms)

In [ ]:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["labels"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

train_dataloader = DataLoader(
    dataset["train"], shuffle=True, collate_fn=collate_fn, batch_size=32
)
eval_dataloader = DataLoader(
    dataset["validation"], collate_fn=collate_fn, batch_size=32
)

## 🗻 **Optimizer**

As optimizer, we use AdamW or Adam or ..., making sure that biases and layer normalization are not considered during optimization if we fine-tune. 

In [ ]:
def get_parameter_names(model, forbidden_layer_types):
    result = []
    for name, child in model.named_children():
        result += [
            f"{name}.{n}"
            for n in get_parameter_names(child, forbidden_layer_types)
            if not isinstance(child, tuple(forbidden_layer_types))
        ]
    # Add model specific parameters (defined with nn.Parameter) since they are not in any child.
    result += list(model._parameters.keys())
    return result

In [ ]:
decay_parameters = get_parameter_names(model, [nn.LayerNorm])
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if n in decay_parameters],
        "weight_decay": cn_train.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
        "weight_decay": 0.,
    },
]

**choose optimizer:**

Two implementations of AdamW, AdamP, Adam and Shampoo. In the experiments Adam is used. For fine-tuning model.parameters() must be replaced with optimizer_grouped_parameters, since we don't want to train layernorm etc. If training from scratch, leave it.

In [ ]:
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr = 4e-5,
    betas = (0.9, 0.999),
    eps = 1e-08
)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=4e-5, weight_decay = 0.001)

In [ ]:
import torch_optimizer

optimizer = torch_optimizer.AdamP(
    model.parameters(), 
    lr=4e-5, 
    delta=0.1,
    wd_ratio=0.1,
    nesterov=False,
    weight_decay=5e-3, 
    eps=1e-08
)

In [ ]:
import torch_optimizer

optimizer = torch_optimizer.Shampoo(
    model.parameters(), 
    lr=cn_train.learning_rate
)

In [ ]:
optimizer = torch.optim.Adam(
    model.parameters(), 
    lr = cn_train.learning_rate
)

**Linear scheduler, can be bypassed**

In [ ]:
# adapted from https://github.com/huggingface/transformers/blob/main/src/transformers/optimization.py 

from torch.optim.lr_scheduler import LambdaLR
from torch.optim import Optimizer

def get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps):
    def lr_lambda(step: int):
        if step < num_warmup_steps:
            return float(step) / float(max(1, num_warmup_steps))
        return max(
            0.0, float(num_training_steps - step) / float(max(1, num_training_steps - num_warmup_steps))
        )

    return LambdaLR(optimizer, lr_lambda)

In [ ]:
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    cn_train.warmup_steps,
    int(len(train_dataloader) * cn_train.epochs)
)

## 🚋 **Train** and **Evaluate**

**Loss**

In [ ]:
from torch.nn import CrossEntropyLoss
loss_fct = CrossEntropyLoss()

**Metrics**

In [ ]:
train_metric = load_metric("accuracy")
eval_metric = load_metric("accuracy")

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

**Steps**

In [ ]:
def train_step(batch):
    batch = {k: v.to(device) for k, v in batch.items()}
    

    # pass the batch to the model 
    outputs = model(**batch)

    # compute the loss function 
    loss = loss_fct(outputs.view(-1, cn_args.num_labels), batch["labels"].view(-1))
    loss.backward()

    optimizer.step()
    # lr_scheduler.step()
    optimizer.zero_grad()

    # update the metric
    predictions = outputs.argmax(dim=-1)
    train_metric.add_batch(
        predictions = predictions,
        references = batch["labels"],
    )
    return loss

def test_step(batch):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
        loss = loss_fct(outputs.view(-1, cn_args.num_labels), batch["labels"].view(-1))
    
    # get the scores for the batch 
    # tensor {0, 1} * batch_size
    predictions = outputs.argmax(dim=-1)

    eval_metric.add_batch(
        predictions = predictions, 
        references = batch["labels"]
    )
    return loss

**Training**: 

Adapting here our Keras training script to PyTorch, it's quite the same. 

In [ ]:
#@title ConvNeXt Tensorboard
%reload_ext tensorboard
%tensorboard --logdir runs

In [ ]:
def train_and_evaluate(epochs, train_dataloader, eval_dataloader):
    num_train_examples = len(train_dataloader)
    num_eval_examples = len(eval_dataloader)

    global_eval_accuracy = 0.
    epochs_progress_bar = tqdm(range(epochs), desc=f"Epoch: (1/{epochs})", position=0)
    step = 0
    for epoch in range(epochs):
        print(f"\nEpoch: {epoch+1}")

        # ======================== Training ===============================
        model.train()
        train_step_progress_bar = tqdm(total=num_train_examples, desc="Training", position=1, leave=True)

        for _, batch in enumerate(train_dataloader):
            loss = train_step(batch)
            writer.add_scalar("Train Loss", float(loss), step)
            step += 1

            desc = (
                f"Training loss at step {step}: {round(float(loss), 4)}\n"
            )
            train_step_progress_bar.desc = desc
            train_step_progress_bar.update(1)  

        train_acc = train_metric.compute()
        train_acc = train_acc["accuracy"]
        print(f"Train accuracy: {round(train_acc, 4)}")
        writer.add_scalar("Train Accuracy", train_acc, epoch)

        # ======================== Evaluation ==============================
        model.eval()
        eval_step_progress_bar = tqdm(total=num_eval_examples, desc="Eval", position=2, leave=True)
        eval_losses = []
        for _, batch in enumerate(eval_dataloader):
            eval_loss = test_step(batch)
            eval_losses.append(eval_loss.cpu().numpy())

            eval_step_progress_bar.desc = desc
            eval_step_progress_bar.update(1)
        
        eval_loss = np.mean(eval_losses)
        writer.add_scalar("Eval Loss", eval_loss, epoch)
        print("Eval loss: ", round(float(eval_loss), 4))

        eval_acc = eval_metric.compute()
        eval_acc = eval_acc["accuracy"]
        print(f"Eval accuracy: {round(eval_acc, 4)}")
        writer.add_scalar("Eval Accuracy", eval_acc, epoch)

        if eval_acc > global_eval_accuracy:
            global_eval_accuracy = eval_acc
            torch.save(model, "convnext_pytorch.bin")

        epochs_progress_bar.update(1) 
    writer.flush()

In [ ]:
train_and_evaluate(
    cn_train.epochs,
    train_dataloader, 
    eval_dataloader
)

In [ ]:
#@title delete **model** - clean **cache**
del model
torch.cuda.empty_cache()

In [ ]:
#@title Train with **Huggingface** Trainer

!python run_image_classification.py \
    --model_name_or_path ./convnext-base-224-22k-1k \
    --train_dir ./dataset/Cat_vs_Dogs_split/train \
    --validation_dir ./dataset/Cat_vs_Dogs_split/val \
    --output_dir ./trainer_convnext/ \
    --remove_unused_columns False \
    --ignore_mismatched_sizes True \
    --do_train \
    --do_eval \
    --learning_rate 2e-5 \
    --num_train_epochs 8 \
    --per_device_train_batch_size 32 \
    --per_device_eval_batch_size 32 \
    --logging_strategy steps \
    --logging_steps 50 \
    --evaluation_strategy epoch \
    --save_strategy epoch \
    --load_best_model_at_end True \
    --save_total_limit 3 \
    --seed 1337